# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [1]:
! rm -r /kaggle/working/*

# Analyze the tender

Locate the tender document - in PDF format.

In [2]:
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/tender_wind.pdf -P /kaggle/working

--2024-11-20 11:45:28--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/tender_wind.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495170 (484K) [application/octet-stream]
Saving to: '/kaggle/working/tender_wind.pdf'

tender_wind.pdf     100%[===================>] 483.56K  --.-KB/s    in 0.03s   

2024-11-20 11:45:29 (18.0 MB/s) - '/kaggle/working/tender_wind.pdf' saved [495170/495170]



In [3]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tender_wind.pdf') else True
if use_kaggle_input_tender:
    tender_file_path = '/kaggle/input/tenders/tender_wind.pdf'
else:
    tender_file_path = '/kaggle/working/tender_wind.pdf'

print(f"The file {tender_file_path} will be used as the tender")

The file /kaggle/working/tender_wind.pdf will be used as the tender


Installing required Python packages to analyze the tender - in PDF format.

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.6 MB/s eta 0:00:00


Extract information from the tender.
The output will be a text.

In [5]:
import PyPDF2

# Initialize an empty string to store the full text of the PDF
tender_info = ""

# Open the PDF file
with open(tender_file_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    
    # Iterate over all pages and append their text to tender_info
    for page in reader.pages:
        tender_info += page.extract_text()  # Append page text to solar_tender

# Print the extracted text
# print(tender_info)

# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [6]:
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/companies_info.json -P /kaggle/working

--2024-11-20 11:45:40--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/companies_info.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3067142 (2.9M) [text/plain]
Saving to: '/kaggle/working/companies_info.json'

companies_info.json 100%[===================>]   2.92M  --.-KB/s    in 0.04s   

2024-11-20 11:45:40 (69.2 MB/s) - '/kaggle/working/companies_info.json' saved [3067142/3067142]



Locate the JSON file containing the companies' information.

In [7]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info.json') else True
if use_kaggle_input_companies:
    companies_info_file_path = '/kaggle/input/companies-websites/companies_info.json'
else:
    companies_info_file_path = '/kaggle/working/companies_info.json'

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

The file /kaggle/working/companies_info.json will be used for the information regarding the companies


Define a small function to read the information about the companies - in JSON format.

In [8]:
import json

def read_companies_info(jsonFilePath: str) -> dict:
    
    with open(jsonFilePath, "r") as f:
        data = json.load(f)
    return data

Load the companies' information by using the defined function.

In [9]:
companies_info = read_companies_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [10]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest')

chat = model.start_chat()

# example how to send prompts
# response = chat.send_message('Hi! What about competing on Kaggle?')
# print(response.text)

## Define the system and user prompts

In [11]:
system_prompt = "You are an experienced technical sales manager. You are given the content of websites of companies with their portfolio for products and solutions. Use the content of the websites to provide detailed technical answers based on tender requirements and user prompt."
# print(f"The system prompt is:\n{system_prompt}")

In [12]:
user_prompt = f"""

I want to build the whole plant as per tender technical requirements {tender_info}.

1. Analyse and identify all the technical requirements in the tender.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results.

3. Calculate an affinity score in % of company [SIEMENS] and [HITACHI] based on the match of results in point 2.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# print(f"The user prompt is:\n{user_prompt}")

## Check how many tokens are needed to process the prompts

In [13]:
# Split the prompt into words (tokens)
tokens = user_prompt.split()

# Count the number of tokens
num_tokens = len(tokens)
print(f"Number of tokens (words): {num_tokens}")

Number of tokens (words): 420633


## Generate a response

In [14]:
want_a_response = False # disable if willing to save a notebook but the Gemini quota has been exhausted

if want_a_response:
    response = chat.send_message(system_prompt + user_prompt)
    print(response.text)

## Check how many tokens have been used so far

In [15]:
if want_a_response:
    usage_metadata = response.usage_metadata
    print(f"Token count: {usage_metadata.total_token_count}")